In [8]:
import datetime, time
import json
import urllib.parse
import pandas as pd
import configparser
from pprint import pprint
from tqdm import tqdm, tqdm_notebook

import sqlite3
tqdm.pandas(tqdm_notebook)

In [3]:
con = sqlite3.connect("data/pkp/main.db")
df = pd.read_sql_query("SELECT * from PKP", con)
df = df.set_index('doi')

In [10]:
def parse_json(row):
    try:
        row['am_response'] = json.loads(row['am_response'])
    except:
        pass

    try:
        row['fb_og_object'] = json.loads(row['fb_og_object'])
    except:
        pass
    
    try:
        row['fb_engagement'] = json.loads(row['fb_engagement'])
    except:
        pass
    return row
df = df.progress_apply(parse_json, axis=1)

168228it [00:29, 5715.08it/s]                            


In [11]:
def extract_values(row):
    if row['fb_og_object']:
        row['fb_og_id'] = row['fb_og_object']['id']
    else:
        row['fb_og_id'] = None
    
    if row['fb_engagement']:
        row['fb_share_count'] = row['fb_engagement']['share_count']
        row['fb_reaction_count'] = row['fb_engagement']['reaction_count']
        row['fb_comment_plugin_count'] = row['fb_engagement']['comment_plugin_count']
        row['fb_comment_count'] = row['fb_engagement']['comment_count']
    else:
        row['fb_share_count'] = None
        row['fb_reaction_count'] = None
        row['fb_comment_plugin_count'] = None
        row['fb_comment_count'] = None
    
    if row['am_response']:
        try:
            row['am_id'] = row['am_response']['altmetric_id']
        except:
            row['am_id'] = None

        try:
            row['am_score'] = row['am_response']['score']
        except:
            row['am_score'] = None

        try:
            row['am_share_count'] = row['am_response']['counts']['facebook']['posts_count']
        except:
            row['am_share_count'] = None
    else:
        row['am_id'] = None
        row['am_score'] = None
        row['am_share_count'] = None
    
    return row
df = df.progress_apply(extract_values, axis=1)

168228it [16:27, 170.43it/s]                            


In [20]:
df.loc[df['doi_resolve_status'] == 200].describe()

,fb_share_count,fb_reaction_count,fb_comment_plugin_count,fb_comment_count,am_id,am_score,am_share_count
count,18235.000000,18235.000000,18235.0,18235.000000,8.323000e+03,8883.000000,3516.000000
mean,11.669866,267.010584,0.0,0.888621,9.921038e+06,3.104076,1.733220
std,67.735013,1713.017845,0.0,5.658603,8.207641e+06,15.551626,5.119671
min,0.000000,0.000000,0.0,0.000000,1.044180e+05,0.000000,1.000000
25%,0.000000,0.000000,0.0,0.000000,3.032828e+06,0.250000,1.000000
50%,0.000000,0.000000,0.0,0.000000,6.592005e+06,0.750000,1.000000
75%,1.000000,0.000000,0.0,0.000000,1.594465e+07,2.600000,2.000000
max,449.000000,11408.000000,0.0,210.000000,3.061007e+07,954.936000,276.000000


In [ ]:
df[['timestamp',
    'doi_resolve_status', 'doi_resolve_error', 'doi_url', 'fb_response_error', 'am_response_error',
    'fb_og_id', 'fb_share_count', 'fb_reaction_count', 'fb_comment_plugin_count', 'fb_comment_count',
    'am_id', 'am_score', 'am_share_count']].to_csv("PKP_temp.csv")